In [ ]:
using JuMP
using Gurobi
using CSV
using DataFrames
using Graphs
using MetaGraphsNext

General structure:

Use a dictionary to store all parameters

Use a directed graph to represent the order of preference

We can use an extra slack to activate

Make sure the problem with all constraints is infeasible

Three methods to obtain a solution:

1. Give weights to constraints (slack variables controlling how much we violate each) in the objective.
We can have a method that takes in the parameter dictionary, and returns a problem.

2. Iterate over all possible combinations of constraints, find the best set of constraints.
It's probably the most complicated part...

3. Use integer optimization to find the best set of constraints to satisfy

In [ ]:
# Let's use a dictionary to store all the data
data = Dict()

# Data example, as a portfolio optimization problem with 5 assets. Actual data should come from the OR database
data["mean_return"] = [0.1, 0.2, 0.3, 0.4, 0.5]
data["standard_deviation"] = [1.0, 2.0, 3.0, 4.0, 5.0]
data["covariance_matrix"] = [1.0 0.1 0.2 0.3 0.4;
                             0.1 1.0 0.1 0.2 0.3;
                             0.2 0.1 1.0 0.1 0.2;
                             0.3 0.2 0.1 1.0 0.1;
                             0.4 0.3 0.2 0.1 1.0]
# Data example, we have extra parameters for constraints
data["expected_return"] = 0.2
data["maximum_uncertainty"] = 0.1
data["max_invest_in_each_asset"] = 0.5
# Basic parameters taken from the data
data["n_assets"] = length(data["mean_return"])

# Let's use a dictionary to store all constraints
constraints = Dict()
# For each constraint, we need to define weight and functions to add and remove the constraint
# It's a lot of work, but I didn't find a better way to do it
constraints["expected_return"] = Dict(
    "weight" => 0.1, # weight for the objective function, only used in the minimize weighted violation method

    # this function takes an optimization model and the data, and returns the model with the plain constraint added
    #Note that the model must have proper variables defined, otherwise this function will fail
    "add_plain_constraint" => function(model, data)
        @constraint(model, expected_return,
                    sum(invest_portion[i]*data["mean_return"][i] for i in 1:data["n_assets"]) >= data["expected_return"],
                    base_name="expected_return",
                    )
        return model
    end,
    # this function rmoves the constraint from the model. useful for the graph search algorithm
    "remove_plain_constraint" => function(model, data)
        try
            delete(model, expected_return)
            deregister(model, expected_return)
        catch e
            println("Constraint expected_return not found")
        end
        return model
    end,

    # this function takes an optimization model and the data, and returns the model with the constraint added
    #Here we also require the violation slack variable, `violation`, is alreay defined in the model
    "add_constraint_with_violation" => function(model, data)
        @constraint(model,
                    sum(invest_portion[i]*data["mean_return"][i] for i in 1:data["n_assets"]) >= data["expected_return"] - violation["expected_return"],
                    base_name="expected_return"),
        return model
    end,

    # this function takes an optimization model and the data, and returns the model with the constraint added
    #Here we require the binary variable, `satisfied`, indicating if the constraint is required to be satisfied (1) or not (0) is already defined in the model
    "add_constraint_with_binary" => function(model, data)
        expected_return_binary = @variable(model, base_name="expected_return_binary")
        large_constant = 10 * maximum(data["mean_return"]) # a large constant for this constraint
        @constraint(model,
                    sum(invest_portion[i]*data["mean_return"][i] for i in 1:data["n_assets"]) >= data["expected_return"] - (1-satisfied["expected_return"])*large_constant,
                    base_name="expected_return",
                    )
        return model
    end,
)
# another constraint, the variance
constraints["variance_in_return"] = Dict(
    "weight" => 0.2,

    "add_plain_constraint" => function(model, data)
        @constraint(model, variance_in_return,
                    sum(invest_portion[i]*invest_portion[j]*data["standard_deviation"][i]*data["standard_deviation"][i]*data["covariance_matrix"][i,j] for i in 1:data["n_assets"], j in 1:data["n_assets"]) <= data["maximum_uncertainty"]^2,
                    base_name="variance_in_return",
                    )
        return model
    end,
    "remove_plain_constraint" => function(model, data)
        try
            delete(model, variance_in_return)
            deregister(model, variance_in_return)
        catch e
            println("Constraint variance_in_return not found")
        end
        return model
    end,

    "add_constraint_with_violation" => function(model, data)
        @constraint(model,
                    sum(invest_portion[i]*invest_portion[j]*data["standard_deviation"][i]*data["standard_deviation"][i]*data["covariance_matrix"][i,j] for i in 1:data["n_assets"], j in 1:data["n_assets"]) <= data["maximum_uncertainty"]^2 + violation["variance_in_return"],
                    base_name="variance_in_return",
                    )
        return model
    end,

    "add_constraint_with_binary" => function(model, data)
        variance_in_return_binary = @variable(model, base_name="variance_in_return_binary")
        large_constant = 10 * maximum(data["standard_deviation"])^2 # a large constant for this constraint
        @constraint(model,
                    sum(invest_portion[i]*invest_portion[j]*data["standard_deviation"][i]*data["standard_deviation"][j]*data["covariance_matrix"][i,j] for i in 1:data["n_assets"], j in 1:data["n_assets"]) <= data["maximum_uncertainty"]^2 + (1-satisfied["variance_in_return"])*large_constant,
                    base_name="variance_in_return",
                    )
        return model
    end,
)

    

# Let's use a list to encode the preferences
preferenece = [
    ("expected_return", "variance_in_return"), # this line means the constraint is more important than variance_in_return
    ("expected_return", "max_invest_in_each_asset"), # this line means the constraint is more important than max_invest_in_each_asset
    ("max_invest_in_each_asset", "variance_in_return"), # this line means the constraint is more important than variance_in_retur.
    # more lines can be added here
]
;

In [ ]:
# Let's use Julia's Graphs package to represent the preferences as a graph
# Data on the vertices will be the (name, distance to dymmy vertex) tuple
priority_graph = MetaGraph(DiGraph(); label_type=Symbol, vertex_data_type=Tuple{String, Int64}, edge_data_type=String)

add_vertex!(priority_graph, :Dummy, ("Dummy", 0))
for (name, _) in constraints
    add_vertex!(priority_graph, Symbol(name), (name, 1))
end

add_vertex!(priority_graph, :max_invest_in_each_asset, ("max_invest_in_each_asset", 1)) # Not in the constraints dictionary yet

for (v1, v2) in preferenece
    add_edge!(priority_graph, Symbol(v1), Symbol(v2), "preference")
end

# calculate the transitive reduction of the graph
priority_graph = transitivereduction(priority_graph)

collect(edges(priority_graph))

The following parts are free test fields.
Not important.

In [ ]:
string_test = "expected_return"
typeof(string_test)

In [ ]:
# Test the slack varaible method

test_problem = Model(Gurobi.Optimizer)

test_dic = Dict(
    "a" => 1,
    "b" => 2,
)

@variable(test_problem, x[1:10] >= 0)
@variable(test_problem, slack_for_active[1:10])
@variable(test_problem, dummy[keys(test_dic)] >= 0)

@objective(test_problem, Max, sum(x[i] for i in 1:10))

@constraint(test_problem, [i=1:10], x[i] <= 10 + slack_for_active[i], base_name="maybe_active")
@constraint(test_problem, sum(x[i] for i in 1:10) <= 20, base_name="total")

optimize!(test_problem)
test_problem

In [ ]:
for i=1:10
    fix(slack_for_active[i], 0)
end

optimize!(test_problem)
test_problem


In [ ]:
println(is_valid(test_problem, constraint_by_name(test_problem, "maybe_active[1]")))
cons = constraint_by_name(test_problem, "maybe_active[1]")
delete(test_problem, cons)
unregister(test_problem, :cons)
println(is_valid(test_problem, cons))

cons = constraint_by_name(test_problem, "total")
println(is_valid(test_problem, cons))
delete(test_problem, cons)
unregister(test_problem, :cons)
println(is_valid(test_problem, cons))

optimize!(test_problem)

In [ ]:
@constraint(test_problem, x[1] <= 10 + slack_for_active[1], base_name="maybe_active[1]")

optimize!(test_problem)

test_problem

In [ ]:
optimize!(test_problem)

In [ ]:
for i=1:10
    unfix(slack_for_active[i])
end

optimize!(test_problem)
test_problem
